<a href="https://colab.research.google.com/github/alditaufik137/Dicoding/blob/main/Submission_MochAldiTaufik_PaperRockScissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SUBMISSION CNN PAPER ROCK and SCISSORS.

NAME : MOCHAMAD ALDI TAUFIK

E-MAIL : alditaufik137@gmail.com

KELAS : MACHINE LEARNING UNTUK PEMULA (MACHINE LEARNING FOR BEGINNERS)

DATASET: https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip

In [116]:
from google.colab import files
import random as rd
import zipfile, os 
from tqdm import tqdm 
import numpy as np  
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import image
import cv2 
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import confusion_matrix 
import shutil
print(tf.__version__)

2.5.0


In [117]:
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2021-05-26 09:23:35--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

/tmp/rockpapersciss 100%[===================>] 307.92M  1.15MB/s    in 5m 32s  

2021-05-26 09:29:09 (949 KB/s) - ‘/tmp/rockpaperscissors.zip’ saved [322873683/322873683]



In [118]:
# melakukan ekstraksi pada file zip
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


In [119]:
base_dir = '/tmp/rockpaperscissors'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors')

In [120]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')> 0.98):
      print("\n98 % Accurated, Stop Training!")
      self.model.stop_training = True

callbacks = myCallback()

In [121]:
if not os.path.exists(train_dir):

  os.mkdir(train_dir)

if not os.path.exists(validation_dir):

  os.mkdir(validation_dir)

In [122]:
train_rock = os.path.join(train_dir, 'rock')
train_paper = os.path.join(train_dir, 'paper')
train_scissors = os.path.join(train_dir, 'scissors')

In [123]:
validation_rock = os.path.join(validation_dir, 'rock')
validation_paper = os.path.join(validation_dir, 'paper')
validation_scissors = os.path.join(validation_dir, 'scissors')

In [124]:
#training
if not os.path.exists(train_rock) :
  os.mkdir(train_rock)

if not os.path.exists(train_paper) :
  os.mkdir(train_paper)

if not os.path.exists(train_scissors) :
  os.mkdir(train_scissors)

#validation
if not os.path.exists(validation_rock) :
  os.mkdir(validation_rock)

if not os.path.exists(validation_paper) :
  os.mkdir(validation_paper)

if not os.path.exists(validation_scissors) :
  os.mkdir(validation_scissors)

In [125]:
train_rock_dir, validation_rock_dir = train_test_split(os.listdir(rock_dir), test_size = 0.4)
train_paper_dir, validation_paper_dir = train_test_split(os.listdir(paper_dir), test_size = 0.4)
train_scissors_dir, validation_scissors_dir = train_test_split(os.listdir(scissors_dir), test_size = 0.4)

In [126]:
for file in train_rock_dir:
  shutil.copy(os.path.join(rock_dir, file), os.path.join(train_rock, file))
for file in train_paper_dir:
  shutil.copy(os.path.join(paper_dir,file), os.path.join(train_paper,file))
for file in train_scissors_dir:
  shutil.copy(os.path.join(scissors_dir,file), os.path.join(train_scissors,file))
for file in validation_rock_dir:
  shutil.copy(os.path.join(rock_dir, file), os.path.join(validation_rock,file))
for file in validation_paper_dir:
  shutil.copy(os.path.join(paper_dir,file), os.path.join(validation_paper,file))
for file in validation_scissors_dir:
  shutil.copy(os.path.join(scissors_dir,file), os.path.join(validation_scissors,file))

In [127]:
print(os.listdir('/tmp/rockpaperscissors/rps-cv-images'))

['rock', 'scissors', 'paper', 'README_rpc-cv-images.txt']


In [128]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    zoom_range = 0.1,
                    width_shift_range=0.2,  
                    height_shift_range=0.2,
                    fill_mode='nearest',
                    validation_split=0.4) 
                    
 
test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2, 
                    zoom_range = 0.1, 
                    width_shift_range=0.2,  
                    height_shift_range=0.2,
                    fill_mode='nearest',
                    validation_split=0.4) 

In [129]:
train_generator = test_datagen.flow_from_directory(
        train_dir,  
        target_size=(150, 150),
        batch_size= 32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size= 32, 
        class_mode='categorical')

Found 2171 images belonging to 3 classes.
Found 2002 images belonging to 3 classes.


In [130]:
#model sequential
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
# menggunakan 'softmax' untuk klasifikasi 3 kelas
    tf.keras.layers.Dense(3, activation='softmax')
])


model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [131]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 15, 15, 128)      

In [ ]:
# latih model dengan model.fit
history = model.fit(
    train_generator,
    steps_per_epoch = 41, 
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 27, 
    verbose =2,
      callbacks=[callbacks]
)

Epoch 1/20
41/41 - 169s - loss: 1.1477 - accuracy: 0.3864 - val_loss: 1.0784 - val_accuracy: 0.4213
Epoch 2/20
41/41 - 79s - loss: 0.9813 - accuracy: 0.5340 - val_loss: 0.8868 - val_accuracy: 0.4988
Epoch 3/20
41/41 - 79s - loss: 0.6174 - accuracy: 0.7582 - val_loss: 0.4157 - val_accuracy: 0.8356
Epoch 4/20
41/41 - 79s - loss: 0.4726 - accuracy: 0.8187 - val_loss: 0.3804 - val_accuracy: 0.8565
Epoch 5/20
41/41 - 79s - loss: 0.3680 - accuracy: 0.8699 - val_loss: 0.2249 - val_accuracy: 0.9363
Epoch 6/20
41/41 - 79s - loss: 0.2955 - accuracy: 0.9059 - val_loss: 0.1122 - val_accuracy: 0.9699
Epoch 7/20
41/41 - 79s - loss: 0.1923 - accuracy: 0.9419 - val_loss: 0.1762 - val_accuracy: 0.9491
Epoch 8/20


In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():

  #Prediksi Gambar
  path = fn 
  img = image.load_img(path, target_size =(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=32)

  #input array numpy 
  scissors  = "[[0. 0. 1.]]"
  paper     = "[[1. 0. 0.]]"
  rock      = "[[0. 1. 0.]]"

  #menghitung hasil array tersebut 

  result = np.array_str(classes) 

  #Mengubah perhitungan hasil bahasa mesin ke dalam bahasa manusia(human)
  if result == paper :
     print("Kertas")
  elif result == rock :
     print("Batu")
  elif result == scissors :
     print("Gunting")
  else :
     print("[[%PAPER. %ROCK. %SCISSORS]]")